Importando as bibliotecas e dependências necessárias:

In [1]:
# Bibliotecas necessárias
import os
import pandas as pd
import sqlite3
import time
import logging

/home/jvcampos/projects/etl-voos-anac/.venv/lib/python3.11/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


Carregando as bases de dados:

In [2]:
# Carregamento das bases de dados

# Base de dados voos ANAC
df_ocorrencias = pd.read_csv('../data/raw/V_OCORRENCIA_AMPLA.csv', sep=';', encoding='utf-8', skiprows=1)

# Base de dados municipios IBGE
df_municipios = pd.read_csv('../data/raw/municipios.csv', sep=';', encoding='latin1')

# Base de dados ocorrências CENIPA
df_cenipa = pd.read_excel('../data/raw/cenipa.xlsx')

In [3]:
# Configuração para exibir todos os dados e colunas
pd.set_option('display.max_columns', None)   # mostra todas as colunas
pd.set_option('display.max_colwidth', None)  # mostra todo o conteúdo da célula (mesmo que seja longo)

In [4]:
df_municipios.head()

,CÓDIGO DO MUNICÍPIO - TOM,CÓDIGO DO MUNICÍPIO - IBGE,MUNICÍPIO - TOM,MUNICÍPIO - IBGE,UF
0,1,1100106,GUAJARÁ-MIRIM,Guajará-Mirim,RO
1,2,1100379,ALTO ALEGRE DOS PARECIS,Alto Alegre dos Parecis,RO
2,3,1100205,PORTO VELHO,Porto Velho,RO
3,4,1100452,BURITIS,Buritis,RO
4,5,1100122,JI-PARANÁ,Ji-Paraná,RO


In [5]:
df_ocorrencias.head()

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Hora_da_Ocorrencia,Municipio,UF,Regiao,Descricao_do_Tipo,ICAO,Latitude,Longitude,Tipo_de_Aerodromo,Historico,Matricula,Categoria_da_Aeronave,Operador,Tipo_de_Ocorrencia,Fase_da_Operacao,Operacao,Danos_a_Aeronave,Aerodromo_de_Destino,Aerodromo_de_Origem,Lesoes_Fatais_Tripulantes,Lesoes_Fatais_Passageiros,Lesoes_Fatais_Terceiros,Lesoes_Graves_Tripulantes,Lesoes_Graves_Passageiros,Lesoes_Graves_Terceiros,Lesoes_Leves_Tripulantes,Lesoes_Leves_Passageiros,Lesoes_Leves_Terceiros,Ilesos_Tripulantes,Ilesos_Passageiros,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
0,25043,202019110,CARAJÁS SERVIÇOS DE LOCAÇÃO LTDA EPF,Incidente,2020-06-17,22:10,ALTO ALEGRE,RR,Norte,OUTROS,SJYG,"2,765","-62,2214",Privado,"A ORIGEM E DESTINO ERAM DESCONHECIDOS, BEM COMO A FINALIDADE DO VOO. DE ACORDO COM INFORMAÇÕES DO OPERADOR, POR VOLTA DAS 18H10MIN LOCAL O PILOTO TEVE PANE NO GPS DA AERONAVE E SIMULTANEAMENTE ENCONTROU CONDIÇÕES METEOROLÓGICAS ADVERSAS NA ROTA, REALIZANDO UM POUSO DE PRECAUÇÃO NO AERÓDROMO ALTO MUCAJAÍ ( SJYG ), ALTO ALEGRE, RR, ÁREA INDÍGENA YANOMAMI. APÓS O POUSO A AERONAVE FOI DANIFICADA E O PILOTO DETIDO PELOS INDÍGENAS. A AERONAVE NÃO TEVE DANOS DURANTE A REALIZAÇÃO DO POUSO DE PRECAUÇÃO. O TRIPULANTE SAIU ILESO.",PTCZC,TPP,CARAJÁS SERVIÇOS DE LOCAÇÃO LTDA EPF,OTHR,Indeterminada,Voo Privado,Nenhum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,182K,L1P,C182,1270.0,4.0,CESSNA AIRCRAFT,verdadeiro
1,25081,0940/2013,TAP AIR PORTUGAL,Incidente,2013-04-13,17:56,Brasília,DF,Centro-Oeste,COLISÃO NO SOLO,SBBR,"-15,8692","-47,9208",Público,"No dia 13 de abril de 2013 a aeronave A-330 da TAP, procedente da cidade de lisboa, pousou na pista 29L no aeroporto internacional de Brasília, quando ao iniciar o táxi para o pátio de estacionamento, foi orientado pelo Controle de Solo para prosseguir pelas TWY M, R e L4. Às 14:56 (horário local), ao taxiar pela TWY L4, a aeronave colidiu a ponta da asa esquerda contra uma torre de iluminação, provocando danos a winglet da referida asa. Todos os 239 passageiros e 11 tripulantes saíram ilesos.",CSTOF,Estrangeira,TAP AIR PORTUGAL,GCOL,Táxi,Voo Regular,Leve,SBBR,XXXX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,239.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,falso
2,25503,202023366,AZUL,Incidente,2020-10-02,19:02,CAMPINAS,SP,Sudeste,FOGO/FUMAÇA,SBKP,"-23,0069","-47,1344",Público,"A AERONAVE DECOLOU DO AERÓDROMO VIRACOPOS (SBKP), CAMPINAS, SP, COM DESTINO AO AERÓDROMO HERCÍLIO LUZ (SBFL), FLORIANÓPOLIS, SC, A FIM DE TRANSPORTAR PESSOAL, COM CINCO TRIPULANTES E CENTO E TRINTA E NOVE PASSAGEIROS A BORDO.DURANTE A SUBIDA, OCORREU A INDICAÇÃO DE 'DUE MID EBAY SMOKE'. A TRIPULAÇÃO EFETUOU OS PROCEDIMENTOS NO QRH E DECIDIU POR RETORNAR PARA O AERÓDROMO DE ORIGEM.FOI DECLARADA EMERGÊNCIA. O POUSO FOI REALIZADO NORMALMENTE E SEM INTERCORRÊNCIAS.A AERONAVE NÃO TEVE DANOS.OS CINCO TRIPULANTES E OS CENTO E TRINTA E NOVE PASSAGEIROS SAÍRAM ILESOS.",PSAEA,TPR,AZUL LINHAS AEREAS BRASILEIRAS S.A,F-NI,Subida,Voo Regular,Nenhum,SBFL,SBKP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,139.0,NaN,NaN,NaN,ERJ 190-400,L2J,E295,61500.0,143.0,EMBRAER,verdadeiro
3,25623,202021754,JOAO MAURICIO GOTTARDI LOPES,Incidente,2020-10-07,15:30,ARAÇATUBA,SP,Sudeste,CONTATO ANORMAL COM A PISTA,SBAU,"-21,1442","-50,4264",Público,"A AERONAVE DECOLOU DO AERÓDROMO GOVERNADOR JOSÉ RICHA (SBLO), LONDRINA, PR, COM DESTINO AO AERÓDROMO DE ARAÇATUBA (SBAU), ARAÇATUBA, SP, PARA UM VOO PRIVADO, COM UM TRIPULANTE E UM PASSAGEIRO A BORDO. NA FINAL PARA O POUSO, SEGUNDO RELATO DO PILOTO, HOUVE UMA RAJADA DE VENTO E AO TOCAR A PISTA HOUVE O DESPRENDIMENTO DA RODA DA BEQUILHA. A AERONAVE TEVE DANOS LEVES. O PILOTO E O PASSAGEIRO SAÍRAM ILESOS.",PTJBJ,TPP,JOAO MAURICIO GOTTARDI LOPES,ARC,Pouso,Voo Privado,Leve,SBAU,SBLO,NaN,NaN,NaN,NaN,N

In [6]:
df_cenipa.head()

,Link,Data,Matrícula,Classificação,Tipo,Localidade,UF,Aeródromo,Operação,Status
0,https://dedalo.sti.fab.mil.br/ocorrencia/87243,2025-06-17,****,INCIDENTE,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPONENTE,CURITIBA,PR,SBBI,PRIVADA,FINALIZADA
1,https://dedalo.sti.fab.mil.br/ocorrencia/87235,2025-06-13,****,INCIDENTE,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPONENTE,PACARAIMA,RR,SJML,TÁXI AÉREO,FINALIZADA
2,https://dedalo.sti.fab.mil.br/ocorrencia/87239,2025-06-13,****,INCIDENTE,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPONENTE,SALVADOR,BA,SBSV,TÁXI AÉREO,FINALIZADA
3,https://dedalo.sti.fab.mil.br/ocorrencia/87238,2025-06-12,****,INCIDENTE,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPONENTE,RIO DE JANEIRO,RJ,NCAD,TÁXI AÉREO,FINALIZADA
4,https://dedalo.sti.fab.mil.br/ocorrencia/87216,2025-06-10,****,INCIDENTE,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPONENTE,CUIABÁ,MT,SBCY,PRIVADA,FINALIZADA


## Etapa 2 - **Modificando Colunas**

**2.1** - Alterando os nomes das colunas:

In [7]:
df_ocorrencias = df_ocorrencias.rename(columns={
'Numero_da_Ocorrencia': 'NR_OCR', 
'Numero_da_Ficha': 'NR_FI', 
'Operador_Padronizado': 'OP_PDR', 
'Classificacao_da_Ocorrencia': 'CLAS_OCR', 
'Data_da_Ocorrencia':'DT_OCR', 
'Hora_da_Ocorrencia':'H_OCR', 
'Municipio':'MCP', 
'UF':'UF', 
'Regiao':'RG', 
'Descricao_do_Tipo':'DCRI_TP', 
'ICAO':'ICAO',
'Latitude':'LATD', 
'Longitude':'LONG', 
'Tipo_de_Aerodromo':'TP_ADRM', 
'Historico':'HIST', 
'Matricula':'MTCL', 
'Categoria_da_Aeronave':'CATG_ANV', 
'Operador':'OPOR', 
'Tipo_de_Ocorrencia':'TP_OCR', 
'Fase_da_Operacao':'FA_OP', 
'Operacao':'OPCAO', 
'Danos_a_Aeronave':'DAN_ANV', 
'Aerodromo_de_Destino':'ADRM_DSTN', 
'Aerodromo_de_Origem':'ADRM_ORGM', 
'Lesoes_Fatais_Tripulantes':'LS_FTL_TRPL',
'Lesoes_Fatais_Terceiros':'LS_FTL_TCOS',
'Lesoes_Fatais_Passageiros':'LS_FTL_PSGO',
'Lesoes_Graves_Tripulantes':'LS_GV_TRPL',
'Lesoes_Graves_Passageiros':'LS_GV_PSGO',
'Lesoes_Graves_Terceiros':'LS_GV_TCOS',
'Lesoes_Leves_Tripulantes':'LS_LV_TRPL',
'Lesoes_Leves_Passageiros':'LS_LV_PSGO',
'Lesoes_Leves_Terceiros':'LS_LV_TCOS',
'Ilesos_Tripulantes':'IL_TRPL',
'Ilesos_Passageiros':'IL_PSGO',
'Lesoes_Desconhecidas_Tripulantes':'LS_DESC_TRPL',
'Lesoes_Desconhecidas_Passageiros':'LS_DESC_PSGO',
'Lesoes_Desconhecidas_Terceiros':'LS_DESC_TCOS',
'Modelo':'MDL',
'CLS':'CLS',
'Tipo_ICAO':'TP_ICAO',
'PMD':'PMD',
'Numero_de_Assentos':'NR_ASTS',
'Nome_do_Fabricante':'NM_FBRC',
'PSSO':'PSSO',
})

**2.2** - Tipando os tipos de dados das colunas:

In [8]:

# Garante que todas as colunas sejam exibidas
pd.set_option('display.max_columns', None)

# Garante que todas as linhas sejam exibidas (ou ajuste para o número desejado)
pd.set_option('display.max_rows', None)

# Garante que o conteúdo das colunas não seja cortado
pd.set_option('display.max_colwidth', None)

# Exibe as primeiras linhas sem truncamento
linhas_nulas = df_ocorrencias[df_ocorrencias['NR_ASTS'].isna()]
linhas_nulas.head()  # ou apenas linhas_nulas.head() em notebooks


,NR_OCR,NR_FI,OP_PDR,CLAS_OCR,DT_OCR,H_OCR,MCP,UF,RG,DCRI_TP,ICAO,LATD,LONG,TP_ADRM,HIST,MTCL,CATG_ANV,OPOR,TP_OCR,FA_OP,OPCAO,DAN_ANV,ADRM_DSTN,ADRM_ORGM,LS_FTL_TRPL,LS_FTL_PSGO,LS_FTL_TCOS,LS_GV_TRPL,LS_GV_PSGO,LS_GV_TCOS,LS_LV_TRPL,LS_LV_PSGO,LS_LV_TCOS,IL_TRPL,IL_PSGO,LS_DESC_TRPL,LS_DESC_PSGO,LS_DESC_TCOS,MDL,CLS,TP_ICAO,PMD,NR_ASTS,NM_FBRC,PSSO
1,25081,0940/2013,TAP AIR PORTUGAL,Incidente,2013-04-13,17:56,Brasília,DF,Centro-Oeste,COLISÃO NO SOLO,SBBR,"-15,8692","-47,9208",Público,"No dia 13 de abril de 2013 a aeronave A-330 da TAP, procedente da cidade de lisboa, pousou na pista 29L no aeroporto internacional de Brasília, quando ao iniciar o táxi para o pátio de estacionamento, foi orientado pelo Controle de Solo para prosseguir pelas TWY M, R e L4. Às 14:56 (horário local), ao taxiar pela TWY L4, a aeronave colidiu a ponta da asa esquerda contra uma torre de iluminação, provocando danos a winglet da referida asa. Todos os 239 passageiros e 11 tripulantes saíram ilesos.",CSTOF,Estrangeira,TAP AIR PORTUGAL,GCOL,Táxi,Voo Regular,Leve,SBBR,XXXX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,239.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,falso
21,5552,/2009,VIA AER,Acidente,2009-07-07,NaN,NaN,PR,Sul,FALHA OU MAU FUNCIONAMENTO DO MOTOR,NaN,NaN,NaN,NaN,NaN,N8540A,ESTRANGEIRA,VIA AER,SCF-PP,NaN,Voo Privado,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,falso
31,26044,202030691,NaN,Incidente,2020-11-07,09:39,BOITUVA,SP,Sudeste,COLISÃO COM OBSTÁCULO DURANTE A DECOLAGEM E POUSO,Fora de Aeródromo,NaN,NaN,-,"O balão decolou de localidade desconhecida, no município de Boituva, SP, a fim de realizar um voo local, com dois tripulantes e dez passageiros a bordo. Durante o pouso, uma rajada de vento levou o balão em direção a fios de alta tensão próximos ao local, vindo a ocorrer a colisão. O balão não sofreu danos. Todos a bordo saíram ilesos.",*****,Sem matrícula,NaN,CTOL,Pouso,Desconhecida,Nenhum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,verdadeiro
54,27087,202180316,NaN,Acidente,2020-11-08,21:00,ARAPONGAS,PR,Sul,INDETERMINADO,Fora de Aeródromo,"-23,3953","-51,5383",-,"A aeronave decolou da fazenda Lageado, no município de Arapongas, PR, por volta das 21h00min (UTC), a fim de realizar voo local, com um piloto a bordo. Após a decolagem, a aeronave perdeu altura e colidiu contra o solo. A aeronave teve danos substanciais. O tripulante sofreu lesões fatais.",*****,PET,NaN,UNK,Indeterminada,Voo Experimental,Substancial,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,falso
60,25925,202039322,NaN,Incidente Grave,2020-03-01,23:00:00,CUIABÁ,MT,Centro-Oeste,FALHA OU MAU FUNCIONAMENTO DO MOTOR,SBCY,"15,65","-56,1175",Público,"A aeronave decolou do Aeroporto Cidade do Panamá Paitilla (MPMG), Panamá, com destino ao Aeródromo de Viracopos (SBKP), Campinas, SP, a fim de realizar voo privado, com dois pilotos e dois passageiros a bordo.Durante o voo de cruzeiro no FL 450 ocorreram múltiplos maus funcionamentos nos sistemas da aeronave, culminando com a falha em ambos os motores. Durante a descida, ao cruzar o FL 280, os pilotos conseguiram reacender os dois motores. Foi declarada situação de emergência. O pouso foi alternado para o Aeródromo Marechal Rondon (SBCY), Cuiabá, MT. A aeronave pousou normalmente.Não houve danos à aeronave.Todos a bordo saíram ilesos.",N36DA,Estrangeira,NaN,SCF-PP,Cruzeiro,Voo Privado,Nenhum,SBKP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,falso


In [9]:
df_ocorrencias['DT_OCR'] = pd.to_datetime(df_ocorrencias['DT_OCR'], errors='coerce').dt.normalize()

df_ocorrencias['LS_FTL_TRPL'] = pd.to_numeric(df_ocorrencias['LS_FTL_TRPL'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_FTL_TCOS'] = pd.to_numeric(df_ocorrencias['LS_FTL_TCOS'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_FTL_PSGO'] = pd.to_numeric(df_ocorrencias['LS_FTL_PSGO'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_GV_TRPL'] = pd.to_numeric(df_ocorrencias['LS_GV_TRPL'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_GV_PSGO'] = pd.to_numeric(df_ocorrencias['LS_GV_PSGO'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_GV_TCOS'] = pd.to_numeric(df_ocorrencias['LS_GV_TCOS'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_LV_TRPL'] = pd.to_numeric(df_ocorrencias['LS_LV_TRPL'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_LV_PSGO'] = pd.to_numeric(df_ocorrencias['LS_LV_PSGO'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_LV_TCOS'] = pd.to_numeric(df_ocorrencias['LS_LV_TCOS'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['IL_TRPL'] = pd.to_numeric(df_ocorrencias['IL_TRPL'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['IL_PSGO'] = pd.to_numeric(df_ocorrencias['IL_PSGO'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_DESC_TRPL'] = pd.to_numeric(df_ocorrencias['LS_DESC_TRPL'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_DESC_PSGO'] = pd.to_numeric(df_ocorrencias['LS_DESC_PSGO'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_DESC_TCOS'] = pd.to_numeric(df_ocorrencias['LS_DESC_TCOS'], errors='coerce').astype('Int64').fillna(0).astype(int)

# Colunas Latitude e Longitude = LATD e LONG
## Trocando vírgulas por ponto para realizar a tipagem
df_ocorrencias['LONG'] = (
    df_ocorrencias['LONG']
    .astype(str)
    .str.replace(',', '.', regex=False)
)

df_ocorrencias['LONG'] = pd.to_numeric(df_ocorrencias['LONG'], errors='coerce')

df_ocorrencias['LATD'] = (
    df_ocorrencias['LATD']
    .astype(str)                    # Garante que tudo é string
    .str.replace(',', '.', regex=False)  # Troca vírgula por ponto
)

df_ocorrencias['LATD'] = pd.to_numeric(df_ocorrencias['LATD'], errors='coerce')


# df_ocorrencias['NR_ASTS'] = pd.to_numeric(df_ocorrencias['NR_ASTS'], errors='coerce').astype('Int64').fillna(0).astype(int)

df_ocorrencias['PSSO'] = df_ocorrencias['PSSO'].apply(
    lambda x: str(x).strip().lower() == 'verdadeiro'
)

In [10]:
df_ocorrencias.dtypes

NR_OCR                   int64
NR_FI                   object
OP_PDR                  object
CLAS_OCR                object
DT_OCR          datetime64[ns]
H_OCR                   object
MCP                     object
UF                      object
RG                      object
DCRI_TP                 object
ICAO                    object
LATD                   float64
LONG                   float64
TP_ADRM                 object
HIST                    object
MTCL                    object
CATG_ANV                object
OPOR                    object
TP_OCR                  object
FA_OP                   object
OPCAO                   object
DAN_ANV                 object
ADRM_DSTN               object
ADRM_ORGM               object
LS_FTL_TRPL              int64
LS_FTL_PSGO              int64
LS_FTL_TCOS              int64
LS_GV_TRPL               int64
LS_GV_PSGO               int64
LS_GV_TCOS               int64
LS_LV_TRPL               int64
LS_LV_PSGO               int64
LS_LV_TC

## 3 - Preenchendo valores ausentes

In [11]:
# df_ocorrencias[
#     df_ocorrencias["MCP"].isnull() &
#     df_ocorrencias["RG"].isnull() &
#     df_ocorrencias["UF"].isnull() &
#     df_ocorrencias["LATD"].isnull() &
#     df_ocorrencias["LONG"].isnull()
# ]
print('DCRI_TP:')
df_ocorrencias['DAN_ANV'].unique()
# df_ocorrencias[df_ocorrencias['TP_ADRM'] == 'UNK']


# df_ocorrencias.loc[df_ocorrencias['NR_FI'] == '2022445310']


# TRATAR OS VALORES DE nan, Indeterminado, Desconhecido

DCRI_TP:


array(['Nenhum', 'Leve', 'Substancial', nan, 'Destruída', 'Desconhecida'],
      dtype=object)


Valor PMD médio para Balões = 2400.00

CLS não identificado = Desconhecido


In [25]:
# Encontrado na WEB - fonte: https://www.correiobraziliense.com.br/app/noticia/cidades/2013/04/13/interna_cidadesdf%2C360268/aviao-da-tap-bate-em-torre-de-iluminacao-do-aeroporto-jk-nao-ha-feridos.shtml
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25081, 'MDL'] = 'Airbus A330-223'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25081, 'NM_FBRC'] = 'Airbus'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25081, 'NR_ASTS'] = 268
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25081, 'TP_ICAO'] = 'Desconhecido'
# Fonte do PMD:  https://skyteamvirtual.org/fleet/models/delta-air-lines-airbus-a330-223?
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25081, 'PMD'] = 242000.00


# Encontrado na base de dados do CENIPA: https://dedalo.sti.fab.mil.br/ocorrencia/79664
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'MDL'] = 'Balão'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'NM_FBRC'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'NR_ASTS'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'PMD'] = 12
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'PMD'] = 2400.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'LATD'] = -23.2854
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'LONG'] = -47.674
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'OPOR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'OP_PDR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'ADRM_DSTN'] = 'Fora do Aeródromo'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'ADRM_ORGM'] = 'Fora do Aeródromo'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'TP_ICAO'] = 'Desconhecido'


# Encontrado na WEB: https://www.gazetadopovo.com.br/vida-e-cidadania/aeronave-de-pequeno-porte-cai-em-curitiba-bnfeefp83hvbfbs5a6zzp1kjy/
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'MDL'] = 'Bonanza A35'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'NM_FBRC'] = 'Beechcraft'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'NR_ASTS'] = 4
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'MCP'] = 'CURITIBA'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'H_OCR'] = '16:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'LATD'] = -25.55982683577607
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'LONG'] = -49.23159109274909
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'HIST'] = 'Uma aeronave de pequeno porte caiu perto do zoológico de Curitiba na tarde de terça-feira (7), após uma pane no motor. O avião, modelo Bonanza A35, vinha de Londrina com destino ao Aeroporto Afonso Pena. O piloto conseguiu fazer um pouso forçado em área aberta no Parque Iguaçu. Ninguém ficou ferido e houve apenas danos materiais. Os dois ocupantes não precisaram de atendimento médico e não deram declarações à imprensa.'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'FA_OP'] = 'Indeterminada'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'ADRM_DSTN'] = 'Aeroporto Afonso Pena'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'ADRM_ORGM'] = 'Londrina'
# Fonte do PMD: https://www.bonanza.org/aircraft-index/browse-specific-models/bonanza/bonanza-a35/
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'PMD'] = 2400.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'DAN_ANV'] = 'Leve'


# Encontrado na WEB: https://globoplay.globo.com/v/10706102/
# Encontrado no CENIPA: https://dedalo.sti.fab.mil.br/ocorrencia/79874
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 27087, 'DAN_ANV'] = 'Substancial'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 27087, 'TP_ADRM'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 27087, 'OP_PDR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 27087, 'OPOR'] = 'Aerodesportiva'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 27087, 'ADRM_DSTN'] = 'Fora do Aeródromo'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 27087, 'ADRM_ORGM'] = 'Fora do Aeródromo'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 27087, 'MDL'] = 'Outros'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 27087, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 27087, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 27087, 'PMD'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 27087, 'NR_ASTS'] = 2
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 27087, 'NM_FBRC'] = 'Desconhecido'

					
# Encontrado na WEB: https://sistema.cenipa.fab.mil.br/cenipa/paginas/relatorios/rf/pt/RF-N36DA_01_03_2020-Ig.port..pdf
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25925, 'NM_FBRC'] = 'ROSÁRIO OESTE'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25925, 'IL_PSGO'] = 2
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25925, 'IL_TRPL'] = 2
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25925, 'OP_PDR'] = 'Aeronautical Airmotive Modifications INC'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25925, 'OPOR'] = 'Aeronautical Airmotive Modifications INC'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25925, 'ADRM_ORGM'] = 'Aeroporto Marcos A. Gelabert International Airport'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25925, 'MDL'] = 'Gulfstream G‑1159A'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25925, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25925, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 25925, 'PMD'] = 26000.00
# Fonte dos assentos: https://www.flugzeuginfo.net/acdata_php/acdata_g3_en.php
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25925, 'NR_ASTS'] = 19
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25925, 'NM_FBRC'] = 'Gulfstream Aerospace Corporation'


# Encontrado no CENIPA: https://dedalo.sti.fab.mil.br/ocorrencia/79371
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24466, 'MDL'] = 'B744'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24466, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24466, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24466, 'PMD'] = 385553.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24466, 'NM_FBRC'] = 'Boeing'
# Fonte dos assentos: https://doc8643.com/aircraft/B744
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24466, 'NR_ASTS'] = 450

# Só estava faltando esses dados.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26623, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26623, 'TP_ICAO'] = 'Desconhecido'
# Fonte dos assentos: https://www.vansaircraft.com/rv-4/
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26623, 'NR_ASTS'] = 2

# Fonte WEB - https://noticias.uol.com.br/cotidiano/ultimas-noticias/2020/11/17/aviao-bolivia-cai-mato-grosso-piloto-morre.htm?
# Fonte CENIPA - https://dedalo.sti.fab.mil.br/ocorrencia/79678
# Por se tratar de uma aeronave aparentemente boliviana, não se tem informações sobre.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26067, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26067, 'OP_PDR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26067, 'OPOR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26067, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26067, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26067, 'OP_PDR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26067, 'MDL'] = 'Outros'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26067, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26067, 'PMD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26067, 'NR_ASTS'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26067, 'NM_FBRC'] = 'Desconhecido'

# Encontrado no CENIPA: https://dedalo.sti.fab.mil.br/ocorrencia/79751
# Por se tratar de um Aeródromo Privado, as informações foram restritas e o número de matrícula
# para consulta está anonimazado.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26565, 'OP_PDR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26565, 'OPOR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26565, 'MDL'] = 'Outros'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26565, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26565, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26565, 'PMD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26565, 'NR_ASTS'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26565, 'NM_FBRC'] = 'Desconhecido'

# Encontrado na WEB: https://folharegionalnet.com.br/2020/04/12/balao-cai-e-dois-morrem-em-ibitinga/
# Relatório CENIPA: https://sistema.cenipa.fab.mil.br/cenipa/paginas/relatorios/rf/pt/XXXXX.pdf
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24621, 'OP_PDR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24621, 'OPOR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24621, 'ADRM_DSTN'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24621, 'ADRM_ORGM'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24621, 'MDL'] = 'Balão'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24621, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24621, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24621, 'PMD'] = 2400.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24621, 'NR_ASTS'] = 7
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24621, 'IL_TRPL'] = 2
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24621, 'NM_FBRC'] = 'Desconhecido'

# Não encontrado na WEB
# Não encontrado na base de dados do CENIPA (SIPAER)
# Não encontrado nos relatórios finais CENIPA
# CONCLUSÃO: Por se tratar de uma ocorrência do tipo 'Operações no Solo'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24921, 'OP_PDR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24921, 'OPOR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24921, 'MDL'] = 'Outros'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24921, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24921, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24921, 'PMD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24921, 'NR_ASTS'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 24921, 'NM_FBRC'] = 'Desconhecido'

# Encontrado na WEB: https://aeroin.net/avioes-a-29-fumaca-pa-31t-risco-colisao-ar-fortaleza/
# Encontrado nos Relatórios Finais CENIPA: https://sistema.cenipa.fab.mil.br/cenipa/paginas/relatorios/rf/pt/RF-PR-HRM.pdf
# Encontrado na base de dados do CENIPA: https://painelsipaer.cenipa.fab.mil.br/extensions/Sipaer/Ocorrencias.html
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25583, 'OP_PDR'] = 'FORÇA AÉREA BRASILEIRA'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25583, 'CATG_ANV'] = 'Militar'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25583, 'OPOR'] = 'FORÇA AÉREA BRASILEIRA'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25583, 'MDL'] = 'A-29'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25583, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25583, 'TP_ICAO'] = 'Desconhecido'
# Fonte do PMD: https://podermil.wordpress.com/2009/10/27/ficha-tecnica-do-a-29-super-tucano-embraer-emb-314/
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25583, 'PMD'] = 5400.00
# Fonte dos assentos: http://warwingsdaily.com/trainers/embraer-emb-314-super-tucano-a-29/
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25583, 'NR_ASTS'] = 2
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25583, 'NM_FBRC'] = 'EMBRAER'

# Encontrado na base de dados do CENIPA - SIPAER: https://dedalo.sti.fab.mil.br/ocorrencia/79598
# Nenhuma notícia encontrada na WEB.
# Não encontrado nos Relatórios Finais CENIPA.
# CONCLUSÃO: Por se tratar de uma ocorrência fora do Aeródromo e a Aeronave ter sido encontrada carbonizada, não é possível ter informações sobre a aeronave e a operação que estava fazendo.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25724, 'OP_PDR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25724, 'LATD'] = -22.58
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25724, 'LONG'] = -53.0589
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25724, 'TP_ADRM'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25724, 'OPOR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25724, 'OPCAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25724, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25724, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25724, 'MDL'] = 'Outros'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25724, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25724, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25724, 'PMD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25724, 'NR_ASTS'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25724, 'NM_FBRC'] = 'Desconhecido'

# Encontrado nos Relatórios Finais CENIPA: https://sistema.cenipa.fab.mil.br/cenipa/paginas/relatorios/rf/pt/CX-JYN_03_07_2014.pdf
# Não encontrado na base de dados do CENIPA (Por ser uma ocorrência de 2014).
# Não encontrada nenhuma notícia sobre o Acidente.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5951, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5951, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5951, 'ADRM_ORGM'] = 'SUAA'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5951, 'MDL'] = '402B'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5951, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5951, 'TP_ICAO'] = 'Desconhecido'
# Fonte PMD: https://www.flugzeuginfo.net/acdata_php/acdata_cessna401_en.php
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5951, 'PMD'] = 3107.00
# Fonte dos assentos: https://pt.scribd.com/document/331960335/Cessna-402B-pdf
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5951, 'NR_ASTS'] = 8
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5951, 'NM_FBRC'] = 'Cessna Aircraft'

# Encontrado na WEB: https://avherald.com/h?article=47839fd7&
# Não encontrado na base de dados do CENIPA - SIPAER (Por ser uma ocorrência de 2014).
# Não encontrado nos relatórios finais do CENIPA.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5984, 'DCRI_TP'] = 'FALHA OU MAU FUNCIONAMENTO DO MOTOR'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5984, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5984, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5984, 'TP_OCR'] = 'EVAC'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5984, 'ADRM_ORGM'] = 'Paris Charles de Gaulle'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5984, 'MDL'] = 'A330-200'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5984, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5984, 'TP_ICAO'] = 'Desconhecido'
# Fonte do PMD: https://es.wikipedia.org/wiki/Airbus_A330
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5984, 'PMD'] = 233000.00
# Fonte dos assentos: https://www.aerolopa.com/af-332
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5984, 'NR_ASTS'] = 224
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5984, 'NM_FBRC'] = 'Air France Airbus'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na base de dados do CENIPA - SIPAER
# Não encontrado nos Relatórios Finais CENIPA
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5966, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5966, 'MDL'] = 'Outros'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5966, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5966, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5966, 'PMD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5966, 'NR_ASTS'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5966, 'NM_FBRC'] = 'Desconhecido'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na base de dados do CENIPA - SIPAER (Por ser uma ocorrência de 2014).
# Encontrado nos Relatórios Finais CENIPA: https://sistema.cenipa.fab.mil.br/cenipa/paginas/relatorios/rf/pt/pr_tax_19_08_14.pdf
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 6079, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 6079, 'MDL'] = 'PA 34-200'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 6079, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 6079, 'TP_ICAO'] = 'Desconhecido'
# Fonte do PMD: https://en.wikipedia.org/wiki/Piper_PA-34_Seneca
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 6079, 'PMD'] = 1814.00
# Fonte dos assentos: http://planephd.com/wizard/details/556/PIPER-PA-34-200-Seneca-specifications-performance-operating-cost-valuation
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 6079, 'NR_ASTS'] = 5
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 6079, 'NM_FBRC'] = 'PIPER AIRCRAFT'

# Notícia encontrada na WEB: https://noca.com.br/noticia/49180-aviao-cai-e-explode-na-br-135-no-maranhao
# Encontrado nos Relatórios Finais do CENIPA: https://sistema.cenipa.fab.mil.br/cenipa/paginas/relatorios/rf/pt/PR-DTY_18_04_2020-AC.pdf
# Encontrado na base de dados do CENIPA - SIPAER
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11254, 'MCP'] = 'MATÕES DO NORTE'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11254, 'ADRM_DSTN'] = 'Fora do Aeródromo'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11254, 'ADRM_ORGM'] = 'Fora do Aeródromo'
# Fonte dos Assentos: https://pt.wikipedia.org/wiki/Piper_PA-25_Pawnee
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11254, 'NR_ASTS'] = 1

# Não encontrado nenhuma notícia na WEB: Por se tratar de um incidente sem colisões ou feridos, não virou notícia, nem se obtém dados sobre.
# Encontrado na base de dados do CENIPA - SIPAER: https://dedalo.sti.fab.mil.br/ocorrencia/79089
# Não encontrado nos Relatórios Finais - CENIPA. 
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'MCP'] = 'FOZ DO IGUAÇU'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'UF'] = 'PR'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'RG'] = 'Sul'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'LATD'] = -25.5469
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'LONG'] = -54.5882
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'TP_ADRM'] = 'Privado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'FA_OP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'ADRM_DSTN'] = 'NIID'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'ADRM_ORGM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'MDL'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'PMD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'NR_ASTS'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11171, 'NM_FBRC'] = 'Desconhecido'

# Não encontrado nenhuma notícia na WEB.
# Encontrado na base de dados do CENIPA - SIPAER: https://dedalo.sti.fab.mil.br/ocorrencia/79287
# Não encontrado nos Relatórios Finais - CENIPA. 
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11226, 'TP_ADRM'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11226, 'MDL'] = 'A320neo'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11226, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11226, 'TP_ICAO'] = 'A20N'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11226, 'PMD'] = 77000.00
# Fonte dos assentos: https://aircraft.airbus.com/en/aircraft/a320-family/a320neo
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11226, 'NR_ASTS'] = 194
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11226, 'NM_FBRC'] = 'AIRBUS'

# Não encontrado nenhuma notícia na WEB.
# Encontrado na Base de Dados do CENIPA - SIPAER: https://dedalo.sti.fab.mil.br/ocorrencia/79265
# Não encontrado nos Relatórios Finais - CENIPA. 
# Por se tratar de uma Operação Privada, não é possível obter todos os dados.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'OP_PDR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'DCRI_TP'] = 'Indeterminado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'LATD'] = -26.226387
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'LONG'] = -48.509833
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'TP_ADRM'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'OPOR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'TP_OCR'] = 'CTOL'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'FA_OP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'OPCAO'] = 'Privada'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'ADRM_DSTN'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'ADRM_ORGM'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'MDL'] = 'Outros'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'PMD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'NR_ASTS'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'NM_FBRC'] = 'Desconhecido'

# Não encontrado nenhuma notícia na WEB.
# Encontrado na Base de Dados do CENIPA - SIPAER: https://dedalo.sti.fab.mil.br/ocorrencia/79314
# Não encontrado nos Relatórios Finais - CENIPA. 
# Por se tratar de um incidente que ocorreu no momento de decolagem, não repercutiu para uma notícia 
# nem se obtém informações para onde estava indo.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11222, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11222, 'MDL'] = 'B738'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11222, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11222, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11222, 'PMD'] = 70533.00
# Fonte dos assentos: https://en.wikipedia.org/wiki/Boeing_737_Next_Generation
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11222, 'NR_ASTS'] = 189
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11222, 'NM_FBRC'] = 'Boeing'

# Não encontrado nenhuma notícia na WEB.
# Encontrado na Base de Dados do CENIPA - SIPAER: https://dedalo.sti.fab.mil.br/ocorrencia/79211
# Não encontrado nos Relatórios Finais - CENIPA. 
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11206, 'TP_ADRM'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11206, 'ADRM_DSTN'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11206, 'ADRM_ORGM'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11206, 'CLS'] = 'EMB-203'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11206, 'TP_ICAO'] = 'Desconhecido'
# Fonte dos assentos e PMD: https://agricultural.embraer.com/br/pt/ipanema-203
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11206, 'PMD'] = 1900.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11206, 'NR_ASTS'] = 2
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11206, 'MDL'] = 'IPAN'


# Não encontrado nenhuma notícia na WEB.
# Encontrado na Base de Dados do CENIPA - SIPAER: https://dedalo.sti.fab.mil.br/ocorrencia/79331
# Não encontrado nos Relatórios Finais - CENIPA. 
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11353, 'LATD'] = -15.2343
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11353, 'LONG'] = -59.3311
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11353, 'ADRM_DSTN'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11353, 'ADRM_ORGM'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11353, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11353, 'TP_ICAO'] = 'EMB-203'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11353, 'NR_ASTS'] = 2
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11353, 'MDL'] = 'IPAN'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015)
# Encontrado nos Relatórios Finais - CENIPA: https://sistema.cenipa.fab.mil.br/cenipa/paginas/relatorios/rf/pt/n413hb_26_08_2009.pdf
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11329, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11329, 'ADRM_DSTN'] = 'SBSP'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11329, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11329, 'ADRM_ORGM'] = 'SBSP'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11329, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11329, 'TP_ICAO'] = 'Desconhecido'
# Fonte dos assentos e PMD: https://planephd.com/wizard/details/662/HAWKER-BEECHCRAFT-CORP-4000-specifications-performance-operating-cost-valuation
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11329, 'NR_ASTS'] = 10
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11329, 'PMD'] = 1134.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11329, 'MDL'] = 'Beech 4000'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11329, 'NM_FBRC'] = 'Hawker Beechcraft'


# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# Não encontrado nas Recomendações de Segurança.
# A aeronave foi encontrada já pegando fogo, sem nenhum tripulante ou passageiro, não sendo possível
# ser identificado as informações acerca de destinos, modelo, etc.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4493, 'OP_PDR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4493, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4493, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4493, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4493, 'OPOR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4493, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4493, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4493, 'NR_ASTS'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4493, 'PMD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4493, 'MDL'] = 'Outros'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4493, 'NM_FBRC'] = 'Desconhecido'
	
# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# Não encontrado nas Recomendações de Segurança.
# Por se tratar de um Voo Privado não é possível se obter as informações.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'OP_PDR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'H_OCR'] = '00:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'MCP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'UF'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'RG'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'DCRI_TP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'LATD'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'LONG'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'HIST'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'OPOR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'FA_OP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'DAN_ANV'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'NR_ASTS'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'PMD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'MDL'] = 'Outros'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4615, 'NM_FBRC'] = 'Desconhecido'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Encontrado nos Relatórios Finais - CENIPA: https://sistema.cenipa.fab.mil.br/cenipa/paginas/relatorios/rf/pt/pt_wbs_24_01_01.pdf
# Os únicos dados que se tem sobre o ocorrido é a data e Tipo da aeronave sendo estrangeira,
# foi encontrado um relatório do mesmo dia, com uma aeronave estrangeira,
# dito isso, foi utilizado o relatório para preenchimento dos dados ausentes.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'OP_PDR'] = 'Alberto Buttler'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'H_OCR'] = '14:45'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'MCP'] = 'Uberlândia'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'UF'] = 'MG'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'RG'] = 'Sudeste'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'DCRI_TP'] = 'Falha do Motor em Vôo'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'LATD'] = -18.903512
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'LONG'] = -48.324008
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'TP_ADRM'] = 'Privado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'HIST'] = 'O piloto decolou do aeródromo Pousada do Rio Quente-GO com destino a UberabaMG, com etapa de vôo prevista para 40 min. A autonomia da aeronave, neste trecho, era de aproximadamente 01 h e 10 min. Com 25 min de vôo, quando sobrevoava a cidade de Uberlândia, o motor parou de funcionar. Houve o pouso forçado da aeronave sobre um bairro residencial. Na colisão, duas residências e um automóvel foram avariados. A aeronave sofreu danos graves e o piloto saiu ileso.'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'OPOR'] = 'Alberto Buttler'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'FA_OP'] = 'Em rota'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'DAN_ANV'] = 'Substancial'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'ADRM_DSTN'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'ADRM_ORGM'] = 'SWTQ'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'TP_ICAO'] = 'Desconhecido'
# Fonte dos assentos: https://airmart.com/wp-content/uploads/2022/01/Beechcraft-F33A-Bonanza-Performance-and-Specifications.pdf
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'NR_ASTS'] = 4
# Fonte do PMD: https://www.bonanza.org/globalassets/maintenance/gipc-appendix-weight-and-balance.pd
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'PMD'] = 1547.04 
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'MDL'] = 'F33A'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'NM_FBRC'] = 'BEECH AIRCRAFT'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4614, 'IL_TRPL'] = 1
	
# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA (Por não se tratar de uma ocorrência aérea.)
# Por se tratar de uma ocorrência de solo, não é possível obter os dados operacionais,
# pois, mesmo a aeronave tendo colisão com o portão, a mesma ainda conseguiu realizar a operação de voo normalmente,
# dessa forma, não é um caso para se ter notícias nem relatórios de ocorrências.
# Outro fator é que é um voo que ocorreu dos EUA para o Brasil (RJ), então não se tornou uma ocorrência documentada.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4589, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4589, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4589, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4589, 'TP_ICAO'] = 'Desconhecido'
# Foi observado que a matrícula, está preenchida com os dados de matrícula (prefixo) da aeronave,
# dito isso, é possível obter as informações referente ao avião.
# Fonte dos assentos e PMD: https://staralliancevirtual.org/fleet/models/united-boeing-777-222
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4589, 'NR_ASTS'] = 364
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4589, 'PMD'] = 247.200
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4589, 'MDL'] = '777‑222'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 4589, 'NM_FBRC'] = 'Boeing'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA
# Não foi possível obter relatórios ou notícias sobre o caso.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'H_OCR'] = '00:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'MCP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'LATD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'LONG'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'DAN_ANV'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'TP_ICAO'] = 'Desconhecido'
# Foi observado que a matrícula, está preenchida com os dados de matrícula (prefixo) da aeronave,
# dito isso, é possível obter as informações referente ao avião.
# Fonte dos assentos: https://cptdb.ca/wiki/index.php/American_Airlines_767-300ER
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'NR_ASTS'] = 219
# Fonte do PMD: https://pt.wikipedia.org/wiki/Boeing_767
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'PMD'] = 1868.80
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'MDL'] = '767‑323'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3242, 'NM_FBRC'] = 'Boeing'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA (Por não se tratar de uma ocorrência aérea.)
# Por ser uma ocorrência no solo, não é possível obter as informações sobre no CENIPA
# É possível saber a aeronave pois temos a matrícula e operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'H_OCR'] = '00:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'MCP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'DCRI_TP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'LATD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'LONG'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'TP_OCR'] = 'RAMP'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'DAN_ANV'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'ADRM_ORGM'] = 'Não identificado'
# Fonte dos assentos: https://www.privaira.com/fleet/hawker-900xp/
# Fonte do PMD: https://www.globalair.com/aircraft-for-sale/specifications?specid=768
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'NR_ASTS'] = 10
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'PMD'] = 884.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'MDL'] = 'Hawker 900XP'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3191, 'NM_FBRC'] = 'Hawker Beechcraft'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA (Por não se tratar de uma ocorrência aérea.)
# Por se tratar de um incidente no solo, não é possível se obter as ocorrências no CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'H_OCR'] = '00:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'MCP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'LATD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'LONG'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'DAN_ANV'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'ADRM_ORGM'] = 'Não identificado'
# Fonte sobre aeronave: https://www.airfleets.net/ficheapp/plane-a340-606.htm
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'TP_ICAO'] = 'Desconhecido'
# Fonte dos assentos: https://www.seatguru.com/airlines/Iberia/Iberia_Airbus_A340-600_V2.php
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'NR_ASTS'] = 355
# Fonte do PMD: https://aircraft.airbus.com/sites/g/files/jlcbta126/files/2024-11/AC_A340-500_600_1124.pdf
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'PMD'] = 380.000
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'MDL'] = 'Airbus A340‑642'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3179, 'NM_FBRC'] = 'Airbus SAS'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA (Por não se tratar de uma ocorrência aérea.)
# É possível saber a aeronave pois temos a matrícula e operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'H_OCR'] = '00:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'MCP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'LATD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'LONG'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'DAN_ANV'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'ADRM_ORGM'] = 'Não identificado'
# Fonte da aeronave: https://pt.wikipedia.org/wiki/Saab_340
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'TP_ICAO'] = 'Desconhecido'
# Fonte dos assentos: https://www.saab.com/products/saab-340
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'NR_ASTS'] = 34
# Fonte do PMD: https://www.globalair.com/aircraft-for-sale/specifications?specid=779
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'PMD'] = 3400.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'MDL'] = 'Saab 340B'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3176, 'NM_FBRC'] = 'Saab AB'


# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA (Por não se tratar de uma ocorrência aérea.)
# É possível saber a aeronave pois temos a matrícula e operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'H_OCR'] = '00:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'MCP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'DCRI_TP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'LATD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'LONG'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'TP_OCR'] = 'RAMP'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'DAN_ANV'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'ADRM_ORGM'] = 'Não identificado'
# Fonte da aeronave: https://eta-logistics.com/aircraft-specs
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'TP_ICAO'] = 'Desconhecido'
# Fonte dos assentos: https://es.wikipedia.org/wiki/Vuelo_6316_de_Korean_Air_Cargo
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'NR_ASTS'] = 3
# Fonte do PMD: https://www.forecastinternational.com/archive/disp_old_pdf.cfm?ARC_ID=316
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'PMD'] = 90787.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'MDL'] = 'MD‑11F'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3170, 'NM_FBRC'] = 'McDonnell Douglas'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Encontrado nos Relatórios Finais - CENIPA: https://sistema.cenipa.fab.mil.br/cenipa/paginas/relatorios/rf/pt/pt_tpy_08_08_12_1_1.pdf
# OBS: A data está errada, no relatório final consta uma e na base de dados outra, foi feita a alteração seguindo o relatório.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3347, 'DT_OCR'] = '2012-08-08'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3347, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3347, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3347, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3347, 'TP_ICAO'] = 'Desconhecido'
# Fonte dos assentos: https://www.globalair.com/aircraft-for-sale/specifications?specid=719
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3347, 'NR_ASTS'] = 7
# Fonte do PMD: https://www.flyradius.com/embraer-phenom-100/specifications-dimensions
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3347, 'PMD'] = 4750.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3347, 'MDL'] = 'EMB-500'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3347, 'NM_FBRC'] = 'EMBRAER'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3334, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3334, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3334, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3334, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://skyteamvirtual.org/fleet/models/china-cargo-airlines-boeing-777-f6n
# Fonte dos assentos: https://www.aircharter.com/boeing-777f/
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3334, 'NR_ASTS'] = 2
# Fonte do PMD: https://staralliancevirtual.org/fleet/models/ethiopian-boeing-777-f6n
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3334, 'PMD'] = 347815.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3334, 'MDL'] = 'Boeing 777‑F6N'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3334, 'NM_FBRC'] = 'Boeing'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Encontrado nos Relatórios Finais - CENIPA: https://sistema.cenipa.fab.mil.br/cenipa/paginas/relatorios/rf/pt/pr_gfd_09_09_12.pdf
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3455, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3455, 'ADRM_DSTN'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3455, 'ADRM_ORGM'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3455, 'TP_ADRM'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3455, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3455, 'TP_ICAO'] = 'Desconhecido'
# Fonte dos assentos: https://en.wikipedia.org/wiki/Cessna_172
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3455, 'NR_ASTS'] = 4
# Fonte do PMD: https://www.reddit.com/r/flying/comments/zsfd9w/c172n_passengers
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3455, 'PMD'] = 1043.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3455, 'MDL'] = 'CESNNA-172N'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3455, 'NM_FBRC'] = 'Cessna Aircraft'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
# Fonte da aeronave: https://www.modernairliners.com/boeing-767
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3434, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3434, 'TP_ICAO'] = 'SBGR'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3434, 'NR_ASTS'] = 216
# Fonte do PMD: https://skybrary.aero/aircraft/b763
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3434, 'PMD'] = 186880.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3434, 'MDL'] = 'Boeing 767‑332(ER)'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3434, 'NM_FBRC'] = 'Boeing'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
# Por outros dados (LATD, LONG), é possível saber o MCP e TP_ADRM
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3418, 'MCP'] = 'SÃO PAULO'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3418, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3418, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3418, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3418, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://mjet.eu/fleets/gulfstream-g550-bizjet
# Fonte dos assentos: https://www.globalair.com/aircraft-for-sale/specifications?specid=1050&
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3418, 'NR_ASTS'] = 19
# Fonte do PMD: https://www.jetcraft.com/jetstream/2014/07/gulfstream-g550-2003-present
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3418, 'PMD'] = 41277.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3418, 'MDL'] = 'Gulfstream G550'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3418, 'NM_FBRC'] = 'Gulfstream Aerospace'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3414, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3414, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3414, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3414, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://skybrary.aero/aircraft/md11
# Fonte dos assentos: https://www.aircharterserviceusa.com/aircraft-guide/cargo/boeing-usa/boeing-md-11f
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3414, 'NR_ASTS'] = 3
# Fonte do PMD: https://www.bringeraircargo.com/aircraft-types/md11f/
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3414, 'PMD'] = 286000.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3414, 'MDL'] = 'MD‑11F'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3414, 'NM_FBRC'] = 'McDonnell Douglas'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
# Fonte da aeronave: https://www.planespotters.net/airframe/boeing-767-300-n386aa-american-airlines/eyvo9e
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3381, 'TP_OCR'] = 'SCF-NP'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3381, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3381, 'DCRI_TP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3381, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3381, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3381, 'TP_ICAO'] = 'Desconhecido'
# Fonte dos assentos: https://www.seatguru.com/airlines/American_Airlines/American_Airlines_Boeing_767-300_B.php
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3381, 'NR_ASTS'] = 209
# Fonte do PMD: https://www.boeing.com/commercial/aeromagazine/aero_02/textonly/ps02txt.html
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3381, 'PMD'] = 186880.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3381, 'MDL'] = '767-323'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3381, 'NM_FBRC'] = 'Boeing'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3545, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3545, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3545, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3545, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://en.wikipedia.org/wiki/McDonnell_Douglas_MD-11
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3545, 'NR_ASTS'] = 3
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3545, 'PMD'] = 273294.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3545, 'MDL'] = 'MD‑11F'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3545, 'NM_FBRC'] = 'McDonnell Douglas'

# Notícia encontrada na WEB: https://www.baaa-acro.com/crash/crash-mcdonnell-douglas-md-11f-campinas
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Encontrado nos Relatórios Finais - CENIPA. https://sistema.cenipa.fab.mil.br/cenipa/paginas/relatorios/rf/pt/RF_A-501CENIPA2015_N988AR.pdf
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3536, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3536, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3536, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3536, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://en.wikipedia.org/wiki/McDonnell_Douglas_MD-11
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3536, 'NR_ASTS'] = 3
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3536, 'PMD'] = 273294.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3536, 'MDL'] = 'MD-11F'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3536, 'NM_FBRC'] = 'Boeing'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3499, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3499, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3499, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3499, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://www.flightaware.com/resources/registration/N5527L
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3499, 'NR_ASTS'] = 2
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3499, 'PMD'] = 727.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3499, 'MDL'] = 'Cessna 152'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 3499, 'NM_FBRC'] = 'Cessna Aircraft Company'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5836, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5836, 'TP_ADRM'] = 'Privado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5836, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5836, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5836, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://www.flightradar24.com/data/aircraft/cs-ton
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5836, 'NR_ASTS'] = 269
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5836, 'PMD'] =  235000.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5836, 'MDL'] = 'A330‑202'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5836, 'NM_FBRC'] = 'Airbus SAS'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5769, 'TP_ADRM'] = 'Privado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5769, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5769, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5769, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://www.atr-aircraft.com/wp-content/uploads/2020/07/Factsheets_-_ATR_72-600.pdf
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5769, 'NR_ASTS'] = 72
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5769, 'PMD'] = 22800.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5769, 'MDL'] = 'ATR 72‑600'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5769, 'NM_FBRC'] = 'ATR'

# Encontrado na WEB Relatório de Segurança: https://www.gpiaa.gov.pt/upload/membro.id/ficheiros/i006734.pdf
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Encontrado nos Relatórios Finais - CENIPA: https://sistema.cenipa.fab.mil.br/cenipa/paginas/relatorios/rf/pt/cstoj_relatorio.pdf
# (O relatório final, colocaram uma xerox digitaliza do relatório de segurança)
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5920, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5920, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5920, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5920, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5920, 'TP_ICAO'] = 'Desconhecido'
# Fonte dos assentos: http://aviationgeeks1.com5920/airbus-a330-200-perfect-choice-for-regional-airlines
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5920, 'NR_ASTS'] = 406 
# Fonte do PMD: https://aerobook.com/aircraft/a330-200
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5920, 'PMD'] = 242000.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5920, 'MDL'] = 'A330-223'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5920, 'NM_FBRC'] = 'Airbus'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# Não encontrado nas Recomendações de Segurança.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'H_OCR'] = '00:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'MCP'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'LATD'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'LONG'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'FA_OP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'DAN_ANV'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'ADRM_DSTN'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'TP_ICAO'] = 'Desconhecido'
# Não foi encontrado nenhuma aeronave com a matrícula e operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'NR_ASTS'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'PMD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'MDL'] = 'Outros'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1390, 'NM_FBRC'] = 'Desconhecido'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'H_OCR'] = '00:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'MCP'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'LATD'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'LONG'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'FA_OP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'DAN_ANV'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'ADRM_DSTN'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'TP_ICAO'] = 'Desconhecido'
# Não foi encontrado nenhuma aeronave com a matrícula e operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'NR_ASTS'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'PMD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'MDL'] = 'Outros'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1375, 'NM_FBRC'] = 'Desconhecido'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'H_OCR'] = '05:08'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'MCP'] = 'Rio Branco'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'LATD'] = -9.86903
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'LONG'] = -67.89398
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'FA_OP'] = 'Cruzeiro'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'DAN_ANV'] = 'Nenhum'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'ADRM_DSTN'] = 'SBRB'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'TP_ICAO'] = 'Desconhecido'
# Fonte da Aeronave: https://www.airlines-inform.com/commercial-aircraft/boeing-737-300.html
# Fonte dos assentos: https://www.boeing.com/content/dam/boeing/boeingdotcom/company/about_bca/startup/pdf/historical/737-classic-passenger.pdf
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'NR_ASTS'] = 149
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'PMD'] = 62820.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'MDL'] = '737‑300'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1058, 'NM_FBRC'] = 'Boeing'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
# Por se tratar de uma ocorrência de solo, não é possível saber todas informações.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'H_OCR'] = '00:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'MCP'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'LATD'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'LONG'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'FA_OP'] = 'Estacionamento'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'DCRI_TP'] = 'OPERAÇÕES NO SOLO'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'TP_OCR'] = 'RAMP'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'DAN_ANV'] = 'Leve'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://aero-database.com/aircraft/767-300F
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'NR_ASTS'] = 2
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'PMD'] = 186880.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'MDL'] = '767‑346F'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1722, 'NM_FBRC'] = 'Boeing'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
# Por se tratar de uma ocorrência de solo, não é possível saber todas informações.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'H_OCR'] = '07:25'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'MCP'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'LATD'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'LONG'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'FA_OP'] = 'Pouso'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'DAN_ANV'] = 'Leve'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://en.wikipedia.org/wiki/ATR_72
# Fonte dos assentos: https://www.seatlink.com/airlines/flybe/seat-maps/flybe-atr-42-atr-72-atr-seat-map
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'NM_FBRC'] = 'ATR'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'NR_ASTS'] = 78
# Fonte do PMD: https://www.atr-aircraft.com/aircraft-services/aircraft-family/atr-72-600
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'PMD'] = 23000.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1347, 'MDL'] = '72‑600'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
# Por se tratar de uma ocorrência de solo, não é possível saber todas informações.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'H_OCR'] = '16:40'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'MCP'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'LATD'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'LONG'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'FA_OP'] = 'Estacionamento'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'TP_OCR'] = 'RAMP'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'DCRI_TP'] = 'COLISÃO NO SOLO'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'DAN_ANV'] = 'Leve'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'ADRM_DSTN'] = 'LFPG'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://www.airfleets.net/ficheapp/plane-b747-25601.htm
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'NR_ASTS'] = 436
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'PMD'] = 396890.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'MDL'] = '747‑428'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 2010, 'NM_FBRC'] = 'Boeing'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
# Por se tratar de uma ocorrência de solo, não é possível saber todas informações.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'H_OCR'] = '14:30'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'MCP'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'LATD'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'LONG'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'FA_OP'] = 'Estacionamento'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'DCRI_TP'] = 'COLISÃO NO SOLO'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'TP_OCR'] = ' RAMP'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'DAN_ANV'] = 'Leve'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://avpay.aero/company/airspace-auctions/product/2021-air-tractor-at-502b-turboprop-aircraft-for-auction
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'NR_ASTS'] = 2
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'PMD'] = 4264.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'MDL'] = 'AT‑502B'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1304, 'NM_FBRC'] = 'Air Tractor Inc.'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
# Por se tratar de uma ocorrência de solo, não é possível saber todas informações.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'H_OCR'] = '00:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'MCP'] = 'PARÁ DE MINAS'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'LATD'] = -19.84279
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'LONG'] = -44.60041
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'DCRI_TP'] = 'OPERAÇÃO A BAIXA ALTITUDE'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'TP_OCR'] = 'CTOL'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'FA_OP'] = 'Estacionamento'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'DAN_ANV'] = 'Leve'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'ADRM_DSTN'] = 'SNPA'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'ADRM_ORGM'] = 'SNPA'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://en.wikipedia.org/wiki/Cessna_Citation_II
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'NR_ASTS'] = 10
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'PMD'] = 6850.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'MDL'] = '550 Citation II'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1288, 'NM_FBRC'] = 'Cessna Aircraft Company'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
# Por se tratar de uma ocorrência de solo, não é possível saber todas informações.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'H_OCR'] = '00:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'MCP'] = 'CAMPINAS'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'LATD'] = -23.00740
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'LONG'] = -47.13450
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'TP_OCR'] = 'RAMP'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'DCRI_TP'] = 'ESTACIONAMENTO'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'FA_OP'] = 'Indeterminada'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'DAN_ANV'] = 'Leve'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'ADRM_DSTN'] = 'SBKP'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'ADRM_ORGM'] = 'SBKP'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://en.wikipedia.org/wiki/Piper_PA-23
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'NR_ASTS'] = 5
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'PMD'] = 1724.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'MDL'] = 'PA‑23‑150 Apache'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 985, 'NM_FBRC'] = 'Piper Aircraft'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pois temos a matrícula e operador.
# Por se tratar de uma ocorrência de solo, não é possível saber todas informações.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'H_OCR'] = '00:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'MCP'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'LATD'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'LONG'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'TP_OCR'] = 'BIRD'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'DCRI_TP'] = 'COLISÃO COM AVE'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'FA_OP'] = 'Indeterminada'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'DAN_ANV'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'TP_ICAO'] = 'Desconhecido'
# Não encontrado nenhuma aeronave com a matrícula e operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'NR_ASTS'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'PMD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'MDL'] = 'Outros'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 966, 'NM_FBRC'] = 'Desconhecido'

# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# Por se tratar de uma ocorrência de solo, não é possível saber todas informações.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'H_OCR'] = '00:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'MCP'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'LATD'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'LONG'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'TP_ADRM'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'TP_OCR'] = 'SYSTEM'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'DCRI_TP'] = 'FALHA OU MAU FUNCIONAMENTO DO MOTOR'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'FA_OP'] = 'Subida'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'DAN_ANV'] = 'Leve'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'ADRM_DSTN'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'TP_ICAO'] = 'Desconhecido'
# Não foi possível encontrar uma aeronave com a matrícula e operador identificados.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'NR_ASTS'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'PMD'] = 0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'MDL'] = 'Outros'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 884, 'NM_FBRC'] = 'Desconhecido'


# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# Informações preenchidas com base no Histórico da ocorrência.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 879, 'MCP'] = 'Uberaba'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 879, 'LATD'] = -19.977467
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 879, 'LONG'] = -49.009246
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 879, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 879, 'DAN_ANV'] = 'Nenhum'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 879, 'ADRM_DSTN'] = 'SBUR'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 879, 'ADRM_ORGM'] = 'SBAX'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 879, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 879, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 879, 'TP_ICAO'] = 'Desconhecido'
# É possível encontrar a aeronave pela matrícula e operador
# Fonte da aeronave: https://airport-data.com/aircraft/N3045P.html
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 879, 'NR_ASTS'] = 4
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 879, 'PMD'] = 1157.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 879, 'MDL'] = 'R44 II'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 879, 'NM_FBRC'] = 'Robinson Helicopter Company'


# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível encontrar a aeronave pela matrícula e Operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'H_OCR'] = '00:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'MCP'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'DCRI_TP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'LATD'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'LONG'] = -0.0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'TP_ADRM'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'TP_OCR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'FA_OP'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'DAN_ANV'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'ADRM_DSTN'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'ADRM_ORGM'] = 'Não identificado'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://www.planespotters.net/photo/1162870/pp-fxg-aeroclube-do-parana-embraer-emb-711st-corisco-ii-turbo
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'NR_ASTS'] = 4
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'PMD'] = 1315.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'MDL'] = 'EMB‑711ST Corisco II Turbo'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 867, 'NM_FBRC'] = 'Embraer'


# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Encontrado nos Relatórios Finais - CENIPA: https://sistema.cenipa.fab.mil.br/cenipa/paginas/relatorios/rf/pt/n313pc_20_05_10.pdf
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 909, 'H_OCR'] = '19:35'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 909, 'MCP'] = 'Jundiaí'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 909, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 909, 'ADRM_DSTN'] = 'SBJD'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 909, 'ADRM_ORGM'] = 'SSXX'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 909, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 909, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 909, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 909, 'NR_ASTS'] = 2
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 909, 'PMD'] = 5466.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 909, 'MDL'] = 'PA-42-1000'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 909, 'NM_FBRC'] = 'PIPER'


# Encontrado notícia na WEB: https://aeroin.net/boeing-767-da-lan-cargo-segue-voo-para-miami-apos-impacto-com-passaros-em-viracopos/
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# É possível saber a aeronave pela matrícula e operador.
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'H_OCR'] = '02:40'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'MCP'] = 'Campinas'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'DCRI_TP'] = 'COLISÃO COM AVE'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'LATD'] = -23.005836
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'LONG'] = -47.142561
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'TP_OCR'] = 'BIRD'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'FA_OP'] = 'Decolagem'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'DAN_ANV'] = 'Substancial'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'ADRM_DSTN'] = 'MIA'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'ADRM_ORGM'] = 'VCP'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: https://www.planespotters.net/photo/392749/n524la-lan-cargo-boeing-767-346f-wl
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'NR_ASTS'] = 2
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'PMD'] = 186880.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'MDL'] = '767‑346F'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 1885, 'NM_FBRC'] = 'Boeing'


# Não encontrado nenhuma notícia na WEB.
# Não encontrado na Base de Dados do CENIPA - SIPAER (Por se tratar de ter acontecido em um ano < que 2015).
# Não encontrado nos Relatórios Finais - CENIPA.
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'H_OCR'] = '00:00'
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'MCP'] = 'Não identificado'
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'DCRI_TP'] = 'Desconhecido'
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'LATD'] = -0.0
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'LONG'] = -0.0
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'TP_ADRM'] = 'Desconhecido'
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'TP_OCR'] = 'Desconhecido'
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'FA_OP'] = 'Desconhecido'
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'DAN_ANV'] = 'Desconhecido'
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'ADRM_DSTN'] = 'Não identificado'
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'ADRM_ORGM'] = 'Não identificado'
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'ICAO'] = 'Desconhecido'
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'CLS'] = 'Desconhecido'
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'TP_ICAO'] = 'Desconhecido'
# Fonte da aeronave: 
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'NR_ASTS'] = 0
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'PMD'] = 0.0
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'MDL'] = 'Outros'
# df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == , 'NM_FBRC'] = 'Desconhecido'


In [24]:
df_ocorrencias[df_ocorrencias['NR_ASTS'].isnull()]
# df_ocorrencias['NR_ASTS'].isnull().sum()
# df_ocorrencias[df_ocorrencias['NR_OCR'] == 867]

,NR_OCR,NR_FI,OP_PDR,CLAS_OCR,DT_OCR,H_OCR,MCP,UF,RG,DCRI_TP,ICAO,LATD,LONG,TP_ADRM,HIST,MTCL,CATG_ANV,OPOR,TP_OCR,FA_OP,OPCAO,DAN_ANV,ADRM_DSTN,ADRM_ORGM,LS_FTL_TRPL,LS_FTL_PSGO,LS_FTL_TCOS,LS_GV_TRPL,LS_GV_PSGO,LS_GV_TCOS,LS_LV_TRPL,LS_LV_PSGO,LS_LV_TCOS,IL_TRPL,IL_PSGO,LS_DESC_TRPL,LS_DESC_PSGO,LS_DESC_TCOS,MDL,CLS,TP_ICAO,PMD,NR_ASTS,NM_FBRC,PSSO
2428,1885,112/2011,ABSA AEROLINHAS BRASILEIRAS S/A,Incidente,2011-03-14,NaN,NAN,SP,Sudeste,NaN,NaN,NaN,NaN,NaN,"Na decolagem do N524LA de VCP para MIA, por volta das 02:40Z o mesmo foi restrito a AGLO, BETEL a 5500ft. Quando efetuava curva para interceptar a Radial 274 de CPN foi ouvido um estampido no pára-brisa. Logo em seguida, foi detectada pela tripulação, a marca de um pássaro no canto inferior direito do pára-brisa frontal. Como o impacto do pássaro foi de baixo para cima, e não houve dano aparente à aeronave, a mesma prosseguiu normalmente para MIA.",N524LA,ESTRANGEIRA,ABSA AEROLINHAS BRASILEIRAS S/A,NaN,NaN,Voo Regular,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,False
2534,1493,205/2011,TAP AIR PORTUGAL,Incidente,2011-04-02,NaN,NAN,BA,Nordeste,FALHA OU MAU FUNCIONAMENTO DO MOTOR,NaN,NaN,NaN,NaN,"Voo TAP 0186 matricula CSTOG com 242 passageiros a bordo procedente do aeroporto do Galeão(SBGL) e destino Lisboa em Portugal alternou Aeroporto de Salvador, pista 10/28 devido pane no motor 02, a mesma pousou às 01:03h e entrou na posição 10 do patio 03 às 01:13h. A aeronave foi escoltada pelo SCI após seu pouso até a posição de estacionamento no patio 3 onde permaneceu para manutenção. Quantidade de combustível 44.000L.",CSTOG,ESTRANGEIRA,TAP AIR PORTUGAL,SCF-PP,NaN,Voo Regular,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,False
2536,1490,57/2011,POLICIA MILITAR DO DISTRITO FEDERAL,Ocorrência Anormal,2011-04-01,NaN,NAN,RJ,Sudeste,NaN,NaN,NaN,NaN,NaN,AERONAVE FUROU O PNEU ESQUERDO TRASEIRO DURANTE O POUSO,PRLNN,ADD,POLÍCIA MILITAR DO DISTRITO FEDERAL,NaN,NaN,Operação Pública,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,True
2556,1154,327/2010,AEROCRAFT INTERNACIONAL INC,Incidente,2010-08-26,NaN,NAN,MT,Centro-Oeste,CONTATO ANORMAL COM A PISTA,NaN,NaN,NaN,NaN,"O COMANDANTE DA AERONAVE AO RECEBER A INSTRUÇÃO DA CONTROLADORA DO GNA-AT, PARA POUSO NO AERÓDROMO SBAT- ALTA FLORESTA -MT, PELA CABECEIRA 22 , SENDO ASSIM INSTANTE DEPOIS RECEBEU INSTRUÇÕES PARA ALTERNAR A CABECEIRA OPOSTA DENOMINADA 04, ATENDENDO-A, PROSSEGUIU AS INSTRUÇÕES. RELATA O COMANDANTE QUE AS INSTRUÇÕES RECEBIDAS FORAM DEVIDO A CONCENTRAÇÃO DE FUMAÇA NA REGIÃO SENDO ASSIM DIFICULTANDO A VISIBILIDADE PARA OPERAÇÃO DE POUSO NA CABECEIRA 22. DURANTE A COMUNICAÇÃO ENTRE CONTROLADORA E COMANDANTE HOUVE UMA CONFUSÃO QUE OCASIONOU UMA DIVISÃO NA SUA CONCENTRAÇÃO.RELATA O COMANDANTE QUE INSTANTES DEPOIS DURANTE O PROCEDIMENTO DE POUSO, PELA FALTA DE VISIBILIDADE OCORREU UMA ILUSÃO ÓPTICA CAUSADA PELA FUMAÇA, SENDO QUE A PISTA CABECEIRA 04 A 22 TEM UM DECLIVE PRONUNCIADO E POR MOTIVO DO MESMO LEVOU-O A EFETUAR O ARREDONDAMENTO DA AERONAVE PARA POUSO MAIS ALTO QUE O NECESSÁRIO. TOCANDO OS TRENS DE POUSO PRINCIPAIS NO SOLO E EM SEGUIDA A AERONAVE FLUTUOU E TOCOU O PNEU DO TREM DE POUSO DE NARIZ HAVENDO UM TRAVAMENTO NO MESMO, DEVIDO A DEFORMAÇÃO NO EIXO DA RODA LEVANDO A AERONAVE A MOVIMENTOS ALTERNADOS; OCORRENDO A COLISÃO DAS PONTAS DAS PÁS DE HÉLICES CONTRA O SOLO CAUSANDO AVARIAS DAS MESMAS",N4161D,ESTRANGEIRA,AEROCRAFT INTERNACIONAL INC,ARC,NaN,Voo Privado,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,False
2757,1446,516/2010,TAM,Incidente,2010-11-30,NaN,NAN,BA,Nordeste,NaN,NaN,NaN,NaN,NaN,"A AERONAVE PT-MZX EFETUANDO O VOO JJ3153, ETAPA SBGL/SBSV DURANTE O POUSO, AO TOCAR NA PISTA HOUVE COLISÃO COM PÁSSARO NA REGIÃO DO RADOME.",PRMZX,TPR,TAM,NaN,NaN,Voo Regular,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,True
2769,1434,514/2010,TAM,Incidente,2010-11-26,NaN,NAN,RS,Sul,NaN,NaN,NaN,NaN,NaN,"A AERONAVE PT-MZV EFETUANDO O VOO JJ304

In [14]:
pd.set_option('display.max_columns', None)   # mostra todas as colunas
pd.set_option('display.max_colwidth', None)  # mostra todo o conteúdo da célula (mesmo que seja longo)

# Converta para sets, eliminando nulos e garantindo que tudo seja string
todos_municipios = set(df_municipios['MUNICÍPIO - TOM'].dropna().astype(str).str.strip().str.upper())
municipios_incidentes = set(df_ocorrencias['MCP'].dropna().astype(str).str.strip().str.upper())

# Encontre os valores exclusivos de cada conjunto
municipios_so_em_um = (todos_municipios ^ municipios_incidentes)

# Ordenar os que estão apenas em df_ocorrencias
municipios_somente_em_df_incidentes = sorted(municipios_incidentes - todos_municipios)

# Exibindo os municípios que não estão contidos na base de dados do IBGE
# print(municipios_somente_em_df_incidentes)

# Excluindo valores
df_ocorrencias = df_ocorrencias[~df_ocorrencias['MCP'].isin([
    'AGUASCALIENTES, MÉXICO',
    'ARGENTINA',
    'CANCUN, MÉXICO',
    'CANCÚN, MÉXICO',
    'CAPE TOWN',
    'CHITOSE, JAPÃO',
    'GUANAJUATO',
    'CIDADE DO MÉXICO, MÉXICO',
    'LISBOA',
    'MAR DEL PLATA',
    'MISSIONES',
    'ESTADOS UNIDOS',
    'FORA DO BRASIL',
    'GUANAJUATO, MÉXICO',
    'KUALA LUMPUR, MALÁSIA',
    'LISBOA/POTUGAL',
    'LISBOA ,PORTUGAL',
    'MADRID',
    'MAR DEL PLATA, ARGENTINA',
    'MIAMI',
    'MISSIONES,ARG',
    'MONTEVIDEO',
    'MORELOS, MÉXICO',
    'PARIS',
    'PEDRO JUAN CABALLERO',
    'SENEGAL, ÁFRICA',
    'SILAO, GUANAJUATO, MÉXICO',
    'SOMÁLIA, ÁFRICA',
])]

# Reset Index depois de remover os valores
df_ocorrencias = df_ocorrencias.reset_index(drop=True) # Resetar o índice

# Alterando os municípios

df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 11225, 'MCP'] = 'São Francisco do Sul'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 42556, 'MCP'] = 'Salvador'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 38388, 'MCP'] = 'Petrópolis'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 38368, 'MCP'] = 'Santa Isabel'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 7975, 'MCP'] = 'Tabatinga'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 7136, 'MCP'] = 'Belo Horizonte'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 7300, 'MCP'] = 'Uberlândia'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 8505, 'MCP'] = 'Peixoto de Azevedo'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 39089, 'MCP'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 39089, 'UF'] = 'SP'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 39089, 'RG'] = 'Sudeste'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 35706, 'MCP'] = 'Rondolândia'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'AEROPORTO DE GUARULHOS', 'MCP'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'AEROPORTO DE SÃO JOSÉ DOS CAMPOS', 'MCP'] = 'São José dos Campos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'AGUA BOA', 'MCP'] = 'Água boa'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ÁGUAS CLARAS', 'MCP'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ALIANÇA DOS TOCANTINS', 'MCP'] = 'Aliança do Tocantins'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ANAPÓLIS', 'MCP'] = 'Anápolis'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ARACAJÚ', 'MCP'] = 'Aracaju'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ARIPUAÑA', 'MCP'] = 'Aripuanã'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ASSIS CHATEAUBRIAN', 'MCP'] = 'Assis Chateaubriand'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BALNEARIO CAMBORIU', 'MCP'] = 'Balneário Camboriú'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BARRA DO GARCAS', 'MCP'] = 'Barra do Garças'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BASE AÉREA DE SANTOS', 'MCP'] = 'Santos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BELA VISTA DO PARAISO', 'MCP'] = 'Bela Vista do Paraíso'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BELO HORIZONE', 'MCP'] = 'Belo Horizonte'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BIRITIBA MIRIM', 'MCP'] = 'Biritiba-Mirim'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BLUMENAL', 'MCP'] = 'Blumenau'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BOA VISTA DOS RAMOS', 'MCP'] = 'Boa Vista do Ramos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BOM JESUS DE GOIÁS', 'MCP'] = 'Bom Jesus'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BONOPÓLIS', 'MCP'] = 'Bonópolis'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BRAISLÂNDIA', 'MCP'] = 'Brasilãndia'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BRASILIA', 'MCP'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BRASÌLIA', 'MCP'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BRASÍLIA,', 'MCP'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BRASÍLIADF', 'MCP'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAIXIAS DO SUL', 'MCP'] = 'Caxias do Sul'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMOINAS', 'MCP'] = 'Campinas'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPINA GRANDE |', 'MCP'] = 'Campina Grande'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPO DE MARTE'] = 'São Paulo'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPO DOS GOYTACAZES', 'MCP'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPO GRANDEMS', 'MCP'] = 'Campo Grande'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPO GRNADE', 'MCP'] = 'Campo Grande'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPO MORÃO', 'MCP'] = 'Campo Mourão'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPPINAS', 'MCP'] = 'Campinas'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPOS', 'MCP'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPOS DOS GOITACAZES', 'MCP'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPOS DOS GOYTACAS', 'MCP'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPOS DOS GOYTACASES', 'MCP'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAPÃO DO CANOA', 'MCP'] = 'Capão da Canoa'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CARLÂNDIA', 'MCP'] = 'Indiara'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CASIMIRO E ABREU', 'MCP'] = 'Casimirio de Abreu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CHAPADA DOS GUIMA', 'MCP'] = 'Chapada dos Guimarães'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CIDADE DE UNA', 'MCP'] = 'Una'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'COLIDER', 'MCP'] = 'Colíder'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CONSELHEIRO LAFAYETE', 'MCP'] = 'Conselheiro Lafaiete'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CUARI', 'MCP'] = 'Coari'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CUIABA', 'MCP'] = 'Cuiabá'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CURITBA', 'MCP'] = 'Curitiba'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CURITUBA', 'MCP'] = 'Curitiba'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CURUTIBA', 'MCP'] = 'Curitiba'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ENTRE-IJUÍS', 'MCP'] = 'Entre Ijuis'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ESPINGARDA', 'MCP'] = 'Eliseu Martins'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FLOARIANÓPOLIS', 'MCP'] = 'Florianópolis'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FOZ DO IGUAÇ', 'MCP'] = 'Foz do Iguaçu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'GAURULHOS', 'MCP'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'GAUÍRA', 'MCP'] = 'Guaíra'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'GOIANIA', 'MCP'] = 'Goiânia'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'GUARILHOS', 'MCP'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'GUARULHOSS', 'MCP'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA BOA VISTA, QUIRINÓPOLIS', 'MCP'] = 'Quirinópolis'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA FORTALEZA DE SANTA TEREZINHA', 'MCP'] = 'Santa Terezinha'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA FLORESTA, BRASNORTE', 'MCP'] = 'Brasnorte'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA INHUMAS DO CHAPADÃO', 'MCP'] = 'Uberaba'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA INHUMAS DO CHAPADÃO, MG', 'MCP'] = 'Uberaba'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA NOVA AURORA, CAROLINA', 'MCP'] = 'Carolina'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA NOVA ESPERANÇA', 'MCP'] = 'Balsas'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA NOVA ESPERANÇA, BALSAS', 'MCP'] = 'Balsas'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA SANTA BÁRBARA', 'MCP'] = 'Bom Jesus'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA SANTA BÁRBARA, BOM JESUS DE GOIÁS', 'MCP'] = 'Bom Jesus'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA SANTO INACIO', 'MCP'] = 'Itaqui'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA SANTO INÁCIO – RS', 'MCP'] = 'Itaqui'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA TRES FIGUEIRAS', 'MCP'] = 'Itaqui'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA TRÊS FIGUEIRAS', 'MCP'] = 'Itaqui'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA UBIRATÃ', 'MCP'] = 'Primavera do Leste'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA VIGOLO', 'MCP'] = 'Palmas'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'GRAMADINHO', 'MCP'] = 'Seara'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'HANGAR LÍDER AVIAÇÃO', 'MCP'] = 'Belém'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ILHÈUS', 'MCP'] = 'Ilhéus'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'IGARASSÚ', 'MCP'] = 'Igarassu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ITANHAEM', 'MCP'] = 'Itanhaém'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ITAPORANGA D’ AJUDA', 'MCP'] = "Itaporanga D'Ajuda"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'JACAREPAGUÁ', 'MCP'] = "Rio de Janeiro"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'JERICOACOARA', 'MCP'] = "Jijoca de Jericoacoara"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'JI-PARANÃ', 'MCP'] = "Ji-Paraná"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'JUINA', 'MCP'] = "Juína"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'JUNDIAI', 'MCP'] = "Jundiaí"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'LENÇOIS', 'MCP'] = "Lençóis"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'LENÇOIS PAULISTAS', 'MCP'] = "Lençóis Paulista"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'LENÇOIS PAULISTA', 'MCP'] = "Lençóis Paulista"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'LENÇÓIS PAULISTAS', 'MCP'] = "Lençóis Paulista"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'LONDRINA, PR', 'MCP'] = "Londrina"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'LUÍS EDUARDO MAGAL', 'MCP'] = 'Luís Eduardo Magalhães'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'LUIS EDUARDO MAGALHÃES', 'MCP'] = 'Luís Eduardo Magalhães'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MANUEL URBANO', 'MCP'] = 'Manoel Urbano'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MARICA', 'MCP'] = 'Maricá'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MARILIA', 'MCP'] = 'Marília'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MAÇAMBARÁ', 'MCP'] = 'Maçambara'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MIGRANTINÓPOLIS', 'MCP'] = 'Novo Horizonte do Oeste'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MIRASSOL D´OESTE', 'MCP'] = "Mirassol D'Oeste"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MOGI GUAÇU', 'MCP'] = "Mogi-Guaçu"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MOGI-MIRIM', 'MCP'] = "Mogi Mirim"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MONTES CLAROS,', 'MCP'] = "Montes Claros"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'NAVIRAI', 'MCP'] = "Naviraí"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'NITEROI', 'MCP'] = "Niterói"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'NOVA ARIPUANÃ', 'MCP'] = 'Novo Aripuanã'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'NOVA IGUAÇU', 'MCP'] = 'Nova Iguaçu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'NOVA IGUAÇÚ', 'MCP'] = 'Nova Iguaçu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'NOVA MARINGA', 'MCP'] = 'Nova Maringá'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PALIMIU', 'MCP'] = 'Barcelos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PARATININGA', 'MCP'] = 'Paranatinga'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PARATY', 'MCP'] = 'Parati'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PARAUABEBAS', 'MCP'] = 'Parauapebas'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PEDRA PRETA,', 'MCP'] = 'Pedra Preta'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PONTES DE LACERDA', 'MCP'] = 'Pontes e Lacerda'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PORO VELHO', 'MCP'] = 'Porto Velho'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PORTO ESPERIDÃO', 'MCP'] = 'Porto Esperidião '
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PRAIA DE ENSEADA - SÃO FRANCISCO DO SUL', 'MCP'] = 'São Francisco do Sul'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'QUERENCIA DO NORTE', 'MCP'] = 'Querência do Norte'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PORTO ALEGRE', 'MCP'] = 'Porto Alegre'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PORTO ALEGRE |', 'MCP'] = 'Porto Alegre'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PIRAI DO SUL', 'MCP'] = 'Piraí do Sul'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RECIFE', 'MCP'] = 'Recife'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RECUFE', 'MCP'] = 'Recife'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RIO DE JANEIRO', 'MCP'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RIACHÂO DAS NEVES', 'MCP'] = 'Riachão das Neves'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RIO DE JANERIO', 'MCP'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RIO DE JANEIRO ', 'MCP'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RIO DE  JANEIRO', 'MCP'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RORAINOPOLIS', 'MCP'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['MCP'] == "SANT'ANA DO LIVRAMENTO", 'MCP'] = 'Santana do Livramento'
df_ocorrencias.loc[df_ocorrencias['MCP'] == "SANTA CRUZ DAS PALMAS", 'MCP'] = 'Santa Cruz das Palmeiras'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTANA DO PARAISO', 'MCP'] = 'Santana do Paraíso'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTANA DO PARNAÍBA', 'MCP'] = 'Santana de Parnaíba'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTAREM', 'MCP'] = 'Santarém'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTA TEREZINHA DE JESUS', 'MCP'] = 'Santa Terezinha de Itaipu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTA TEREZINHA DO ITAIPU', 'MCP'] = 'Santa Terezinha de Itaipu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTA VITÓRIA DO PAMAR', 'MCP'] = 'Santa Vitória Do Palmar'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTO ANTONIO DO LEVERGER', 'MCP'] = 'Santo Antônio do Leverger'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTO ANTÔNIO DO LEVERGER', 'MCP'] = 'Santo Antônio do Leverger'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO FELIX DO XINGÚ', 'MCP'] = 'São Felix do Xingu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO FELIX DO XINGÚ', 'MCP'] = 'São Felix do Xingu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO JOSE DOS CAMPOS', 'MCP'] = 'São José dos Campos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO JOSE DO RIO PRETO', 'MCP'] = 'São José do Rio Preto'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO LUIZ', 'MCP'] = 'São Luís'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO LUIZ', 'MCP'] = 'São Luís'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTAÉM', 'MCP'] = 'Santarém'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SLAVADOR', 'MCP'] = 'Salvador'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃ PAULO', 'MCP'] = 'São Paulo'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO FELIX DO ARAGUAIA', 'MCP'] = 'São Félix do Araguaia'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO FELIX DO XINGU', 'MCP'] = 'São Félix do Xingu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO FELIX DO XINGÚ', 'MCP'] = 'São Félix do Xingu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SAO PAULO', 'MCP'] = 'São Paulo'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO JOÃO DEL REY', 'MCP'] = 'São João del Rei'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO JOSE DOS PINHARES', 'MCP'] = 'São José dos Pinhais'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO JOSÉ DOS PINHARES', 'MCP'] = 'São José dos Pinhais'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO JOSÉ DO PINHAL', 'MCP'] = 'São José dos Pinhais'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO JOSÉ DOS CAMOPOS', 'MCP'] = 'São José dos Campos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO LUIS', 'MCP'] = 'São Luís'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SERRRANÓPOLIS', 'MCP'] = 'Serranópolis'
df_ocorrencias.loc[df_ocorrencias['MCP'] == "SÍTIO SÃO JOSÉ, ZONA RURAL DE GLÓRIA D'OESTE", 'MCP'] = "Glória D'Oeste"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'TANGARÁ DAQ SERRA', 'MCP'] = 'Tangaré da Serra'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'TELEMACO BORBA', 'MCP'] = 'Telêmaco Borba'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'TIÊTE', 'MCP'] = 'São Paulo'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'TRAJANO DE MORAIS', 'MCP'] = 'Trajano de Moraes'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'TUPANCI', 'MCP'] = 'São Sepé'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'URBERLÂNDIA', 'MCP'] = 'Uberlândia'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'VARZEA GRANDE', 'MCP'] = 'Várza Grande'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'VERISSÍMO', 'MCP'] = 'Veríssimo'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'VILA BELA DA SANTÍSSIMA', 'MCP'] = 'Vila Bela da Santíssima Trindade'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'VITORIA DA CONQUISTA', 'MCP'] = 'Vitória da Conquista'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'XINGUARÁ', 'MCP'] = 'Xinguara'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'YARA', 'MCP'] = 'Bandeirantes'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'XINGUARÁ', 'MCP'] = 'Xinguara'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ÉRICO - AMAJARI', 'MCP'] = 'Amajari'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ÁREA RURAL', 'MCP'] = 'Palestina'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTO ANTÔNIO DE LEVERGER', 'MCP'] = 'Santo Antônio do Leverger'

In [15]:
df_ocorrencias['MCP'] = df_ocorrencias['MCP'].astype(str).str.upper()

In [16]:
df_ocorrencias[df_ocorrencias['MCP'].isnull()]
df_ocorrencias[df_ocorrencias['MCP'] == 'MIAMI']

,NR_OCR,NR_FI,OP_PDR,CLAS_OCR,DT_OCR,H_OCR,MCP,UF,RG,DCRI_TP,ICAO,LATD,LONG,TP_ADRM,HIST,MTCL,CATG_ANV,OPOR,TP_OCR,FA_OP,OPCAO,DAN_ANV,ADRM_DSTN,ADRM_ORGM,LS_FTL_TRPL,LS_FTL_PSGO,LS_FTL_TCOS,LS_GV_TRPL,LS_GV_PSGO,LS_GV_TCOS,LS_LV_TRPL,LS_LV_PSGO,LS_LV_TCOS,IL_TRPL,IL_PSGO,LS_DESC_TRPL,LS_DESC_PSGO,LS_DESC_TCOS,MDL,CLS,TP_ICAO,PMD,NR_ASTS,NM_FBRC,PSSO


In [17]:
df_municipios['MUNICÍPIO - TOM'] = df_municipios['MUNICÍPIO - TOM'].astype(str).str.strip().str.upper()
# df_municipios[df_municipios['MUNICÍPIO - TOM'] == "VILA CONGONHAS"]
# Pará de Minas
df_municipios[df_municipios['MUNICÍPIO - TOM'].str.contains("CAMPINAS", na=False)]


,CÓDIGO DO MUNICÍPIO - TOM,CÓDIGO DO MUNICÍPIO - IBGE,MUNICÍPIO - TOM,MUNICÍPIO - IBGE,UF
1039,1041,2202109,CAMPINAS DO PIAUÍ,Campinas do Piauí,PI
3712,6291,3509502,CAMPINAS,Campinas,SP
4841,8575,4303806,CAMPINAS DO SUL,Campinas do Sul,RS


In [18]:
df_ocorrencias.head()

,NR_OCR,NR_FI,OP_PDR,CLAS_OCR,DT_OCR,H_OCR,MCP,UF,RG,DCRI_TP,ICAO,LATD,LONG,TP_ADRM,HIST,MTCL,CATG_ANV,OPOR,TP_OCR,FA_OP,OPCAO,DAN_ANV,ADRM_DSTN,ADRM_ORGM,LS_FTL_TRPL,LS_FTL_PSGO,LS_FTL_TCOS,LS_GV_TRPL,LS_GV_PSGO,LS_GV_TCOS,LS_LV_TRPL,LS_LV_PSGO,LS_LV_TCOS,IL_TRPL,IL_PSGO,LS_DESC_TRPL,LS_DESC_PSGO,LS_DESC_TCOS,MDL,CLS,TP_ICAO,PMD,NR_ASTS,NM_FBRC,PSSO
0,25043,202019110,CARAJÁS SERVIÇOS DE LOCAÇÃO LTDA EPF,Incidente,2020-06-17,22:10,ALTO ALEGRE,RR,Norte,OUTROS,SJYG,2.7650,-62.2214,Privado,"A ORIGEM E DESTINO ERAM DESCONHECIDOS, BEM COMO A FINALIDADE DO VOO. DE ACORDO COM INFORMAÇÕES DO OPERADOR, POR VOLTA DAS 18H10MIN LOCAL O PILOTO TEVE PANE NO GPS DA AERONAVE E SIMULTANEAMENTE ENCONTROU CONDIÇÕES METEOROLÓGICAS ADVERSAS NA ROTA, REALIZANDO UM POUSO DE PRECAUÇÃO NO AERÓDROMO ALTO MUCAJAÍ ( SJYG ), ALTO ALEGRE, RR, ÁREA INDÍGENA YANOMAMI. APÓS O POUSO A AERONAVE FOI DANIFICADA E O PILOTO DETIDO PELOS INDÍGENAS. A AERONAVE NÃO TEVE DANOS DURANTE A REALIZAÇÃO DO POUSO DE PRECAUÇÃO. O TRIPULANTE SAIU ILESO.",PTCZC,TPP,CARAJÁS SERVIÇOS DE LOCAÇÃO LTDA EPF,OTHR,Indeterminada,Voo Privado,Nenhum,NaN,NaN,0,0,0,0,0,0,0,0,0,1,0,0,0,0,182K,L1P,C182,1270.0,4.0,CESSNA AIRCRAFT,True
1,25081,0940/2013,TAP AIR PORTUGAL,Incidente,2013-04-13,17:56,BRASÍLIA,DF,Centro-Oeste,COLISÃO NO SOLO,SBBR,-15.8692,-47.9208,Público,"No dia 13 de abril de 2013 a aeronave A-330 da TAP, procedente da cidade de lisboa, pousou na pista 29L no aeroporto internacional de Brasília, quando ao iniciar o táxi para o pátio de estacionamento, foi orientado pelo Controle de Solo para prosseguir pelas TWY M, R e L4. Às 14:56 (horário local), ao taxiar pela TWY L4, a aeronave colidiu a ponta da asa esquerda contra uma torre de iluminação, provocando danos a winglet da referida asa. Todos os 239 passageiros e 11 tripulantes saíram ilesos.",CSTOF,Estrangeira,TAP AIR PORTUGAL,GCOL,Táxi,Voo Regular,Leve,SBBR,XXXX,0,0,0,0,0,0,0,0,0,11,239,0,0,0,Airbus A330-223,NaN,Desconhecido,242000.0,268.0,Airbus,False
2,25503,202023366,AZUL,Incidente,2020-10-02,19:02,CAMPINAS,SP,Sudeste,FOGO/FUMAÇA,SBKP,-23.0069,-47.1344,Público,"A AERONAVE DECOLOU DO AERÓDROMO VIRACOPOS (SBKP), CAMPINAS, SP, COM DESTINO AO AERÓDROMO HERCÍLIO LUZ (SBFL), FLORIANÓPOLIS, SC, A FIM DE TRANSPORTAR PESSOAL, COM CINCO TRIPULANTES E CENTO E TRINTA E NOVE PASSAGEIROS A BORDO.DURANTE A SUBIDA, OCORREU A INDICAÇÃO DE 'DUE MID EBAY SMOKE'. A TRIPULAÇÃO EFETUOU OS PROCEDIMENTOS NO QRH E DECIDIU POR RETORNAR PARA O AERÓDROMO DE ORIGEM.FOI DECLARADA EMERGÊNCIA. O POUSO FOI REALIZADO NORMALMENTE E SEM INTERCORRÊNCIAS.A AERONAVE NÃO TEVE DANOS.OS CINCO TRIPULANTES E OS CENTO E TRINTA E NOVE PASSAGEIROS SAÍRAM ILESOS.",PSAEA,TPR,AZUL LINHAS AEREAS BRASILEIRAS S.A,F-NI,Subida,Voo Regular,Nenhum,SBFL,SBKP,0,0,0,0,0,0,0,0,0,5,139,0,0,0,ERJ 190-400,L2J,E295,61500.0,143.0,EMBRAER,True
3,25623,202021754,JOAO MAURICIO GOTTARDI LOPES,Incidente,2020-10-07,15:30,ARAÇATUBA,SP,Sudeste,CONTATO ANORMAL COM A PISTA,SBAU,-21.1442,-50.4264,Público,"A AERONAVE DECOLOU DO AERÓDROMO GOVERNADOR JOSÉ RICHA (SBLO), LONDRINA, PR, COM DESTINO AO AERÓDROMO DE ARAÇATUBA (SBAU), ARAÇATUBA, SP, PARA UM VOO PRIVADO, COM UM TRIPULANTE E UM PASSAGEIRO A BORDO. NA FINAL PARA O POUSO, SEGUNDO RELATO DO PILOTO, HOUVE UMA RAJADA DE VENTO E AO TOCAR A PISTA HOUVE O DESPRENDIMENTO DA RODA DA BEQUILHA. A AERONAVE TEVE DANOS LEVES. O PILOTO E O PASSAGEIRO SAÍRAM ILESOS.",PTJBJ,TPP,JOAO MAURICIO GOTTARDI LOPES,ARC,Pouso,Voo Privado,Leve,SBAU,SBLO,0,0,0,0,0,0,0,0,0,1,1,0,0,0,180J,L1P,C180,1270.0,6.0,CESSNA AIRCRAFT,True
4,25624,202004387,JOSE LUIZ CUNHA RODRIGUES,Incidente,2020-10-06,15:25,SÃO PAULO,SP,Sudeste,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPONENTE,SBMT,-23.5100,-46.6300,Público,"A AERONAVE DECOLOU DO AERÓDROMO ESTADUAL DE CAMPOS DOS AMARAIS-PREFEITO FRANCISCO AMARAL (SDAM), EM CAMPINAS-SP, PARA VOO PRIVADO, COM PREVISÃO DE PROCEDIMENTOS DE TOQUE E ARREMETIDA NO AERÓDROMO CAMPO DE MARTE (SBMT), EM SÃO PAULO-SP. DURANTE O PROCEDIMENTO DE TOQUE E ARREMETIDA, 